# Learning-Based Edge Offloading Decision

## Abstract
This experiment models a decision-making system that determines whether to execute a computational task **locally** on a device or **offload** it to an edge server.  
A synthetic dataset simulates tasks with varying input sizes, computational costs, network latencies, and CPU loads.  
A neural network is trained to learn offloading decisions based on system and network conditions, mimicking an intelligent edge computing policy.

In [1]:
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

## 1. Synthetic Dataset Generation
The `OffloadDataset` class constructs training data representing computational tasks:
- **Features:**
  - `input_size` — size of data to transfer (KB)
  - `flops` — required computation (FLOPs)
  - `network_latency` — delay in seconds
  - `cpu_load` — current local CPU utilization (0–1)
- **Labels:**
  - `0`: Execute **locally**
  - `1`: **Offload** to edge

The labeling rule compares estimated local execution time vs. offloading time:
- Local time ≈ `flops / (1e8 * (1 - cpu_load))`
- Offload time ≈ `(input_size / 1e6)` + `network_latency`
If offload time + margin < local time → **offload decision (1)**, else **local (0)**.


In [2]:
# Synthetic dataset: each task has features: input_size, required_flops, network_latency, local_cpu_load
class OffloadDataset(Dataset):
    def __init__(self, n=2000, seed=0):
        np.random.seed(seed)
        self.input_size = np.random.uniform(100, 5000, size=(n,))  # e.g. image size in KB
        self.flops = np.random.uniform(1e6, 1e9, size=(n,))  # compute needed
        self.net_lat = np.random.uniform(0.01, 0.2, size=(n,))  # s
        self.cpu_load = np.random.uniform(0.1, 0.9, size=(n,))
        # Label: decision: 0 = local, 1 = offload
        labels = []
        for ins, fl, nl, cl in zip(self.input_size, self.flops, self.net_lat, self.cpu_load):
            # simple rule: offload if local compute slow AND network latency low
            time_local = fl / (1e8 * (1 - cl))  # assume compute rate 1e8 flops/s scaled by load
            time_offload = ins/ (1e6) + nl  # assume transfer rate 1 MB/s
            if time_offload + 0.01 < time_local:
                labels.append(1)
            else:
                labels.append(0)
        self.labels = np.array(labels, dtype=np.int64)
        self.features = np.stack([
            self.input_size, self.flops, self.net_lat, self.cpu_load
        ], axis=1).astype(np.float32)

    def __len__(self):
        return len(self.labels)
    def __getitem__(self, idx):
        return self.features[idx], self.labels[idx]

## 2. Decision Model
The `DecisionModel` is a lightweight feedforward neural network that classifies tasks as **local** or **offload**:
- **Input:** 4 task features  
- **Architecture:**
  - Fully connected (4 → 32 → 16 → 2)
  - ReLU activations
- **Output:** Two logits (for local/offload classification)

This model learns a **policy** mapping system conditions to offloading choices.

---

## 3. Training Procedure
The training process minimizes **Cross-Entropy Loss** using the Adam optimizer:
- **Input:** Feature vectors from the dataset  
- **Target:** Optimal offloading decision (0 or 1)  
- **Metric:** Classification accuracy  

The model is trained over 10 epochs to learn the decision boundary that balances compute load and latency.

In [3]:
class DecisionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(4, 32), nn.ReLU(),
            nn.Linear(32, 16), nn.ReLU(),
            nn.Linear(16, 2)
        )
    def forward(self, x):
        return self.net(x)

def train(model, loader, device):
    model.train()
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
    for x, y in loader:
        x = x.to(device)
        y = y.to(device)
        optimizer.zero_grad()
        logits = model(x)
        loss = loss_fn(logits, y)
        loss.backward()
        optimizer.step()

def test(model, loader, device):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for x,y in loader:
            x = x.to(device)
            y = y.to(device)
            pred = model(x).argmax(dim=1)
            correct += (pred == y).sum().item()
            total += y.size(0)
    return correct / total

## 4. Evaluation
The model is evaluated on held-out test data:
- Accuracy indicates how well the learned policy matches the analytical decision rule.
- A typical model achieves high accuracy (>95%) due to structured synthetic patterns.

---

## 5. Simulation: Decision Inference
The trained model is tested on new synthetic task conditions:
```python
[2000, 1e8, 0.05, 0.2]
[100, 5e6, 0.15, 0.8]
[4000, 5e8, 0.02, 0.4]


In [4]:
def simulate_decisions(model, features, device):
    model.eval()
    decisions = []
    with torch.no_grad():
        x = torch.tensor(features, dtype=torch.float32, device=device)
        logits = model(x)
        decisions = logits.argmax(dim=1).cpu().numpy()
    return decisions

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dataset = OffloadDataset(n=5000)
train_sz = 4000
train_ds, test_ds = torch.utils.data.random_split(dataset, [train_sz, len(dataset)-train_sz])
train_loader = DataLoader(train_ds, batch_size=64, shuffle=True)
test_loader = DataLoader(test_ds, batch_size=64)

model = DecisionModel().to(device)
for epoch in range(10):
    train(model, train_loader, device)
    acc = test(model, test_loader, device)
    print(f"Epoch {epoch}, accuracy: {acc:.4f}")

# Simulate decision making on new tasks
feats_new = np.array([
    [2000, 1e8, 0.05, 0.2],
    [100, 5e6, 0.15, 0.8],
    [4000, 5e8, 0.02, 0.4],
], dtype=np.float32)
decisions = simulate_decisions(model, feats_new, device)
print("Decisions for new tasks (0 = local, 1 = offload):", decisions)


Epoch 0, accuracy: 0.9940
Epoch 1, accuracy: 0.9940
Epoch 2, accuracy: 0.9940
Epoch 3, accuracy: 0.9940
Epoch 4, accuracy: 0.9940
Epoch 5, accuracy: 0.9940
Epoch 6, accuracy: 0.9940
Epoch 7, accuracy: 0.9940
Epoch 8, accuracy: 0.9940
Epoch 9, accuracy: 0.9940
Decisions for new tasks (0 = local, 1 = offload): [1 1 1]
